In [1]:
import numpy as np
import sympy as sp

from sumpy.expansion.diff_op import (
    make_identity_diff_op,
)
from collections import namedtuple
DerivativeIdentifier = namedtuple("DerivativeIdentifier", ["mi", "vec_idx"])

from sumpy.recurrence import _make_sympy_vec, get_reindexed_and_center_origin_on_axis_recurrence

from immutabledict import immutabledict
from sumpy.expansion.diff_op import LinearPDESystemOperator

In [10]:
var = _make_sympy_vec("x", 2)
var_t = _make_sympy_vec("t", 2)
abs_dist = sp.sqrt((var[0]-var_t[0])**2 + (var[1]-var_t[1])**2)
w = make_identity_diff_op(2)

partial_4x = DerivativeIdentifier((4,0), 0)
partial_4y = DerivativeIdentifier((0,4), 0)
partial_2x2y = DerivativeIdentifier((2,2), 0)
biharmonic_op = {partial_4x: 1, partial_4y: 1, partial_2x2y:2}
list_pde = immutabledict(biharmonic_op)

biharmonic_pde = LinearPDESystemOperator(2, (list_pde,))
g_x_y = abs_dist**2 * (sp.log(abs_dist))

n_init, _, r = get_reindexed_and_center_origin_on_axis_recurrence(biharmonic_pde)

derivs = [sp.diff(g_x_y,
                var_t[0], i).subs(var_t[0], 0).subs(var_t[1], 0)
                                        for i in range(8)]

x_coord = np.random.rand()  # noqa: NPY002
y_coord = np.random.rand()  # noqa: NPY002
coord_dict = {var[0]: var[0], var[1]: var[1]}
derivs = [d.subs(coord_dict) for d in derivs]

n = sp.symbols("n")
s = sp.Function("s")

# pylint: disable-next=not-callable
subs_dict = {s(0): derivs[0], s(1): derivs[1], s(2): derivs[2], s(3): derivs[3]}
check = []

assert n_init == 4
max_order_check = 8
for i in range(n_init, max_order_check):
    check.append(r.subs(n, i).subs(subs_dict) - derivs[i])
    # pylint: disable-next=not-callable
    subs_dict[s(i)] = derivs[i]

In [11]:
max_order_check = 8
for i in range(n_init, max_order_check):
    check.append(r.subs(n, i).subs(subs_dict) - derivs[i])
    # pylint: disable-next=not-callable
    subs_dict[s(i)] = derivs[i]

In [18]:
r.subs(n, 4).subs(subs_dict).subs({var[0]: 23, var[1]: 2.3})

-0.00388859878925869

In [19]:
r.subs(n, 4)

-(-2*x0**2 + 6*x1**2)*s(3)/(x0**3 + x0*x1**2) - (-x0**4 + 6*x0**2*x1**2 + 15*x1**4)*s(1)/(x0**7 + 2*x0**5*x1**2 + x0**3*x1**4) + (x0**4 - 6*x0**2*x1**2 - 15*x1**4)*s(2)/(x0**6 + 2*x0**4*x1**2 + x0**2*x1**4)